# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
weather = pd.read_csv("../output_data/cities.csv")
weather


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Mahébourg,75,MU,1614818133,94,-20.4081,57.7000,24.00,1.03
1,1,Arraial do Cabo,0,BR,1614818037,83,-22.9661,-42.0278,25.00,6.17
2,2,Ogaminana,68,NG,1614818134,75,7.5946,6.2248,24.36,2.19
3,3,Hilo,1,US,1614817993,44,19.7297,-155.0900,26.00,5.14
4,4,Coquimbo,75,CL,1614817998,88,-29.9533,-71.3436,17.00,3.60
...,...,...,...,...,...,...,...,...,...,...
537,537,Warrnambool,100,AU,1614818086,91,-38.3833,142.4833,15.56,1.34
538,538,Lake Shasta,1,US,1614818243,14,40.6804,-122.3708,23.00,3.60
539,539,Marrakesh,100,MA,1614818243,72,31.6315,-8.0083,14.00,2.06
540,540,Qaqortoq,75,GL,1614818243,34,60.7167,-46.0333,8.00,20.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#Configure gmaps
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight
locations= weather[["Lat", "Lng"]]
humidity= weather["Humidity"]

In [5]:
#heatmap
figure=gmaps.figure(center=(46.0, -5.0), zoom_level=1)
max_int=np.max(humidity)

#heatlayer
heat_lay=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3 )

#layer 2
figure.add_layer(heat_lay)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_df = weather[(weather['Max Temp'].between(1,12)) & (weather['Wind Speed']<=8) & (weather['Humidity']<=45)]
ideal_df.reset_index(inplace=True)
del ideal_df['index']


ideal_df
 

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hinton,100,CA,1614809210,33,53.4001,-117.5857,6.00,3.09
1,Sheridan,1,US,1614809210,40,44.8333,-106.9173,11.00,0.33
2,Bethel,1,US,1614808716,37,41.3712,-73.4140,9.00,4.12
3,Kerman,0,IR,1614809203,32,30.2832,57.0788,12.00,1.03
4,Brainerd,90,US,1614809250,39,46.3580,-94.2008,6.00,1.58
5,Jalu,0,LY,1614809253,45,29.0331,21.5482,11.78,4.19
6,Quesnel,1,CA,1614809010,45,52.9999,-122.5029,5.00,3.60
7,Broken Hill,0,AU,1614809269,40,-31.9500,141.4333,12.00,2.06
8,Saint Anthony,75,US,1614808789,43,45.0205,-93.2180,9.00,1.73
9,Marathon,1,US,1614809278,42,44.9000,-89.7668,7.00,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=ideal_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Hinton,CA,53.4001,-117.5857,
1,Sheridan,US,44.8333,-106.9173,
2,Bethel,US,41.3712,-73.4140,
3,Kerman,IR,30.2832,57.0788,
4,Brainerd,US,46.3580,-94.2008,
5,Jalu,LY,29.0331,21.5482,
6,Quesnel,CA,52.9999,-122.5029,
7,Broken Hill,AU,-31.9500,141.4333,
8,Saint Anthony,US,45.0205,-93.2180,
9,Marathon,US,44.9000,-89.7668,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Hinton.
Closest hotel in Hinton is Holiday Inn Express & Suites Hinton.
------------
Retrieving Results for Index 1: Sheridan.
Closest hotel in Sheridan is Best Western Sheridan Center.
------------
Retrieving Results for Index 2: Bethel.
Closest hotel in Bethel is Courtyard by Marriott Danbury.
------------
Retrieving Results for Index 3: Kerman.
Closest hotel in Kerman is Sara Hostel.
------------
Retrieving Results for Index 4: Brainerd.
Closest hotel in Brainerd is Holiday Inn Express & Suites Brainerd-Baxter.
------------
Retrieving Results for Index 5: Jalu.
Closest hotel in Jalu is Jalu Hotel.
------------
Retrieving Results for Index 6: Quesnel.
Closest hotel in Quesnel is Billy Barker Casino Hotel.
------------
Retrieving Results for Index 7: Broken Hill.
Closest hotel in Broken Hill is Palace Hotel.
------------
Retrieving Results for Index 8: Saint Anthony.
Closest hotel in Saint Anthony is Hewing Hotel.
------------
Retrieving Results for Ind

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)
# Display figure
figure



Figure(layout=FigureLayout(height='420px'))